**AI & Machine Learning (KAN-CINTO4003U) - Copenhagen Business School | Spring 2025**

***


# Part III: LLM

Please see the description of the assignment in the README file (section 3) <br>
**Guide notebook**: [guides/llm_guide.ipynb](guides/llm_guide.ipynb)


***

<br>

* Note that you should report results using a classification report. 

* Also, remember to include some reflections on your results: how do they compare with the results from Part I, BoW?, and part II, BERT? Are there any hyperparameters or prompting techniques that are particularly important?

* You should follow the steps given in the `llm_guide` notebook

<br>


***

In [48]:
from decouple import config
from ibm_watsonx_ai import APIClient
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models import ModelInference

import pandas as pd
from sklearn.metrics import classification_report 
from tqdm import tqdm
from ibm_watsonx_ai.foundation_models.schema import TextGenParameters

In [49]:
WX_API_KEY = config('WX_API_KEY')

In [50]:
credentials = Credentials(
    url = "https://us-south.ml.cloud.ibm.com",
    api_key = WX_API_KEY
)

client = APIClient(
    credentials=credentials, 
    project_id="0c3df496-9cce-47fa-8c75-2d6a61c758e4"
)

In [51]:
model = ModelInference(
    api_client=client,
    model_id="mistralai/mistral-large",
)

### 1. Load the data

We can load this data directly from [Hugging Face Datasets](https://huggingface.co/docs/datasets/) - The HuggingFace Hub- into a Pandas DataFrame. Pretty neat!

**Note**: This cell will download the dataset and keep it in memory. If you run this cell multiple times, it will download the dataset multiple times.

You are welcome to increase the `frac` parameter to load more data.

In [52]:
splits = {'train': 'data/train-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
# train = pd.read_parquet("hf://datasets/fancyzhx/ag_news/" + splits["train"])
test = pd.read_parquet("hf://datasets/fancyzhx/ag_news/" + splits["test"])

In [53]:
label_map = {
    0: 'World',
    1: 'Sports',
    2: 'Business',
    3: 'Sci/Tech'
}

def preprocess(df: pd.DataFrame, frac = 1e-2, label_map = label_map, seed=42) -> pd.DataFrame:
    return  (
        df
        .assign(label=lambda x: x['label'].map(label_map))
        [lambda df: df['label'].isin(label_map.values())]
        .groupby('label')
        .apply(lambda x: x.sample(frac=frac, random_state=seed))
        .reset_index(drop=True)

    )

# train_df = preprocess(train, frac=0.01)
test_df = preprocess(test, frac=0.1)

# clear up some memory by deleting the original dataframes
# del train
del test

test_df.shape, # train_df.shape, 

((760, 2),)

In [54]:
PARAMS = TextGenParameters(
    temperature=0,              # Higher temperature means more randomness - In this case we don't want randomness
    max_new_tokens=10,          # Maximum number of tokens to generate
    stop_sequences=[".", "\n"], # Stop generating text when these sequences are encountered
)

model = ModelInference(
    api_client=client,
    model_id="mistralai/mistral-large",  # We could also try a larger model!
    params=PARAMS
)

In [55]:
SYSTEM_PROMPT = """You task is to classify news stories into one of five categories

CATEGORIES:
{categories}

Here are some examples of how to classify news stories:

Example 1:
Text: "Apple Inc. reported record-breaking quarterly profits, with iPhone sales exceeding expectations."
Category: Business

Example 2:
Text: "Scientists discover new species of deep-sea creatures in the Pacific Ocean."
Category: Sci/Tech

Example 3:
Text: "Manchester United secures victory in the Champions League final."
Category: Sports

Example 4:
Text: "UN Security Council convenes emergency meeting to address global crisis."
Category: World

TEXT:
{text}

Please assign the correct category to the text. Answer with the correct category and nothing else.

Category:
"""

In [56]:
CATEGORIES = "- " + "\n- ".join(test_df["label"].unique())  # Create a string with all categories

predictions = []

for text in tqdm(test_df["text"]):

    # format the prompt with the categories and the text
    prompt = SYSTEM_PROMPT.format(categories=CATEGORIES, text=text)
    
    # generate the response from the model
    response = model.generate(prompt)

    # extract the generated text from the response
    prediction = response["results"][0]["generated_text"].strip()

    # append the prediction to the list of predictions
    predictions.append(prediction)

100%|██████████| 760/760 [05:45<00:00,  2.20it/s]


In [57]:
print(classification_report(test_df.label, predictions))

              precision    recall  f1-score   support

    Business       0.70      0.89      0.79       190
    Sci/Tech       0.91      0.64      0.75       190
      Sports       0.97      0.96      0.97       190
       World       0.87      0.90      0.89       190

    accuracy                           0.85       760
   macro avg       0.86      0.85      0.85       760
weighted avg       0.86      0.85      0.85       760



The LLM system did a pretty good job at classifying the news articles, achieving a test accuracy of 85%. This is higher than the Bag-of-Words model's accuracy of 83%, but lower than the BERT model's 88%.

Unlike the LLM guide, I used the mistral-large model, which has the highest classification score among the available models on IBM's platform. I also modified the prompt slightly and used few-shot prompting to give the model examples of what news article text could look like in each category. I found that this approach worked better than zero-shot prompting.

The few-shot prompting helped the model classify the news articles more accurately than the 74% baseline mentioned in the LLM guide.


